### **Name: Manuj Kumar Joshi**

### **Status : Intern**

### **Organisation : The Sparks Foundation**

### **Task 5 : Implement a real time face mask detector**


### **Approach: Transfer Learning**

<h3>Step 1 : Importing the Libraries</h3>

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

<h3>Step 2 : Read the Iris Dataset</h3>

In [2]:
dataset=r'F:\Mask_Detection_Dataset\Dataset'
imagePaths=list(paths.list_images(dataset))

In [3]:
data=[]
labels=[]
for i in imagePaths:
    label=i.split(os.path.sep)[-2]
    labels.append(label)
    image=load_img(i,target_size=(224,224))
    image=img_to_array(image)
    image=preprocess_input(image)
    data.append(image)

In [6]:
data=np.array(data,dtype='float32')
labels=np.array(labels)

In [7]:
lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

In [8]:
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [9]:
train_X,test_X,train_Y,test_Y=train_test_split(data,labels,test_size=0.20,random_state=42,stratify=labels)

In [10]:
train_X.shape

(1560, 224, 224, 3)

In [11]:
train_Y.shape

(1560, 2)

In [12]:
test_X.shape

(390, 224, 224, 3)

In [13]:
test_Y.shape

(390, 2)

In [14]:
aug=ImageDataGenerator(rotation_range=20,
                       zoom_range=0.15,
                       width_shift_range=0.2,
                      shear_range=0.15,
                      horizontal_flip=True,
                      vertical_flip=True,
                      fill_mode='nearest')

In [16]:
baseModel=MobileNetV2(weights='imagenet',include_top=False,
                     input_tensor=Input(shape=(224,224,3)))

9412608/9406464 [==============================] - 2s 0us/step


In [17]:
baseModel

In [19]:
baseModel.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [24]:
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name='Flatten')(headModel)
headModel=Dense(128,activation='relu')(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation='softmax')(headModel)
model=Model(inputs=baseModel.input,
           outputs=headModel)

In [25]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [26]:
for layer in baseModel.layers:
    layer.trainable=False

In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [29]:
learning_rate=0.0001
Epochs=20
Batch_size=12

opt=Adam(lr=learning_rate,decay=learning_rate/Epochs)
model.compile(loss='binary_crossentropy',optimizer=opt,
             metrics=['accuracy'])
H=model.fit(
aug.flow(train_X,train_Y,batch_size=Batch_size),
        steps_per_epoch=len(train_X)//Batch_size,
        validation_data=(test_X,test_Y),
         validation_steps=len(test_X)//Batch_size,
         epochs=Epochs
)

model.save('mobilenet_v2.model')

Epoch 1/20
130/130 [==============================] - 106s 758ms/step - loss: 0.5539 - accuracy: 0.7930 - val_loss: 0.1025 - val_accuracy: 0.9897
Epoch 2/20
130/130 [==============================] - 84s 642ms/step - loss: 0.1538 - accuracy: 0.9683
Epoch 3/20
130/130 [==============================] - 81s 620ms/step - loss: 0.0873 - accuracy: 0.9795
Epoch 4/20
130/130 [==============================] - 70s 536ms/step - loss: 0.0704 - accuracy: 0.9814
Epoch 5/20
130/130 [==============================] - 70s 538ms/step - loss: 0.0538 - accuracy: 0.9856
Epoch 6/20
130/130 [==============================] - 72s 555ms/step - loss: 0.0541 - accuracy: 0.9861
Epoch 7/20
130/130 [==============================] - 72s 552ms/step - loss: 0.0324 - accuracy: 0.9937
Epoch 8/20
130/130 [==============================] - 70s 536ms/step - loss: 0.0391 - accuracy: 0.9887
Epoch 9/20
130/130 [==============================] - 70s 538ms/step - loss: 0.0244 - accuracy: 0.9966
Epoch 10/20
130/130 [=========

In [31]:
predict=model.predict(test_X,batch_size=Batch_size)
predict=np.argmax(predict,axis=1)
print(classification_report(test_Y.argmax(axis=1),
                           predict,target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       1.00      1.00      1.00       190
without_mask       1.00      1.00      1.00       200

    accuracy                           1.00       390
   macro avg       1.00      1.00      1.00       390
weighted avg       1.00      1.00      1.00       390



# Final Phase in next sheet